<h1 style="text-align:center;">Tarea 4: Introducción a la computación de alto rendimiento</h1>
<h2 style="text-align:center;">Ángel Fabricio Aguirre Bermúdez - C10152</h2>

## Parte I

In [ ]:
#include <iostream>
#include <complex>
#include <vector>
#include <omp.h>
#include <sys/time.h>

Se incluyen bibliotecas necesarias para:
* Entrada y salida estándar (`iostream`).
* Manejo de números complejos (`complex`).
* Estructuras dinámicas como vectores (`vector`).
* Paralelización con OpenMP (`omp.h`).
* Medición del tiempo de ejecución (`sys/time.h`).

In [ ]:
double seconds(){
  struct timeval tmp;
  double sec;
  gettimeofday( &tmp, (struct timezone *)0 );
  sec = tmp.tv_sec + ((double)tmp.tv_usec)/1000000.0;
  
  return sec;
}

Esta función se utiliza para medir el tiempo de ejecución en segundos entre dos puntos del código.

In [ ]:
int mandelbrot(const std::complex<double> &c, const int &maxits){
  std::complex<double> z = 0;
  int n = 0;
  while(abs(z) <= 2 && n < maxits){
    z = z * z + c;
    ++n;
  }
    
  return n;
}

Esta función determina si un número complejo $c$ pertenece al Conjunto de Mandelbrot. Itera la fórmula:
\begin{equation*}
z_{n+1} = z_n^2 +c
\end{equation*}
Hasta alcanzar una de dos condiciones:
* $|z|>2$ (indica que $c$ no pertenece al conjunto).
* Número máximo de iteraciones (`maxits`).

Retorna el número de iteraciones completadas.

In [ ]:
void print_mat(const std::vector<char> &mat, int &rows, int &cols){
  for(int i = 0; i < rows; ++i){
    for(int j = 0; j < cols; ++j){
      std::cout << mat[(i * cols) + j];
    }
    std::cout << std::endl;
  }
}

Esta función se utiliza para imprimir una matriz almacenada en un vector unidimensional (como si fuera bidimensional). Es usada para representar gráficamente el Conjunto de Mandelbrot en formato ASCII.

Trabajar con un vector permite que cada hilo itere sobre su sección y separar el cálculo de la impresión, para que los hilos no interfieran entre sí.

In [ ]:
int main(){
    int width = 3840; // Resolución horizontal
    int height = 2160; // Resolución vertical
    int max_iter = 1000; // Máximo de iteraciones por punto
    double minX = -2.0, maxX = 1.0; // Límites horizontales del plano complejo
    double minY = -1.0, maxY = 1.0; // Límites verticales del plano complejo
    std::vector<char> resultado(width*height, '.'); // Matriz inicializada con '.'
    int num_procs; // Número de hilos paralelos

Ya en el `main`, se configuran los parámetros de la visualización:
* Resolución de la salida (`width`, `height`).
* Iteraciones máximas para determinar la pertenencia al conjunto.
* Rango del plano complejo.
* Inicializa un vector (`resultado`) en el que se almacenará el gráfico ASCII.


### Subdivisión de las filas del diagrama

In [ ]:
    double time_1 = seconds();
    //Paralelización del algoritmo con base en una subdivision de las filas:
    #pragma omp parallel 
    {
      num_procs = omp_get_num_threads();
      #pragma omp for
      for(int y = 0; y < height; ++y){
        for(int x = 0; x < width; ++x){
          // Mapeo de pixeles a número complejo
          std::complex<double> c( minX + (maxX - minX) * x / width,
                                  minY + (maxY - minY) * y / height );
            
          // Cálculo del número de iteraciones
          int n = mandelbrot(c, max_iter);
  
          // Se guarda un caracter dependiendo del número de iteraciones
          if(n == max_iter){
              resultado[(y * width) + x] = '#'; // Dentro del conjunto de Mandelbrot
          } 
        }
      }
    }
    double time_2 = seconds();

Acá, se itera por cada píxel de la matriz, mapeándolo a un número complejo `c` según las coordenadas en el plano.

Se calcula si `c` pertenece al Conjunto de Mandelbrot usando la función `mandelbrot`.

Además, se usa OpenMP para paralelizar el cálculo, dividiendo las filas (`y`) entre varios hilos.

Los puntos pertenecientes al conjunto son marcados en `resultado`con `'#'`.

Durante el proceso se mide el tiempo de ejecución total para poder graficar la escalabilidad.

In [ ]:
    // print_mat(resultado, height, width);

Se imprime el resultado en la terminal. Está comentado para poder calcular la escalabilidad, pero con una resolución de 155x50 se comprobó que el resultado es el correcto, el mismo que el del código en serie proporcionado:
<div style="text-align: center;">
  <img src="Terminal_mandelbrot.png" style="width: 70%; margin: auto;">
</div>

In [ ]:
    std::cout << "# Num Threads: " << num_procs << std::endl;
    std::cout << "# Time: " << time_2 - time_1 << std::endl;
    return 0;
}

Finalmente se imprime estadísticas del cálculo:
* Número de hilos paralelos usados.
* Tiempo total de ejecución.

La siguiente figura corresponde al gráfico de escalabilidad utilizando hasta 8 hilos para la subdivisión de filas:

<div style="text-align: center;">
  <img src="Tiempo_filas.png" style="width: 80%; margin: auto;">
</div>


### Subdivisión de las columnas del diagrama

In [ ]:
    double time_1 = seconds();
    for(int y = 0; y < height; ++y){
      // Paralelización del algoritmo con base en una subdivision de las columnas:
      #pragma omp parallel
      {
        num_procs = omp_get_num_threads();
        #pragma omp for 
        for(int x = 0; x < width; ++x){
            // Mapeo de pixeles a número complejo
            std::complex<double> c( minX + (maxX - minX) * x / width,
                                    minY + (maxY - minY) * y / height );
            
            // Cálculo del número de iteraciones
            int n = mandelbrot(c, max_iter);
  
            // Se guarda un caracter dependiendo del número de iteraciones
            if(n == max_iter){
                resultado[(y * width) + x] = '#'; // Dentro del conjunto de Mandelbrot
            } 
        }
      }
    }
    double time_2 = seconds();

Acá, el código es el mismo, pero  al usar OpenMP y paralelizar el cálculo, se divide las columnas (`x`), en vez de las filas.

In [ ]:
    // print_mat(resultado, height, width);

    std::cout << "# Num Threads: " << num_procs << std::endl;
    std::cout << "# Time: " << time_2 - time_1 << std::endl;
    return 0;
}

Análogamente, se imprime el resultado en la terminal (de igual manera es el correcto para la resolución menor, mas no se mostrará para evitar redundancia), así como el número de hilos paralelos usados y el tiempo total de ejecución.

La siguiente figura corresponde al gráfico de escalabilidad utilizando hasta 8 hilos para la subdivisión de columnas:

<div style="text-align: center;">
  <img src="Tiempo_cols (1).png" style="width: 80%; margin: auto;">
</div>

A modo  de análisis de ambos gráficos: estos presentan una disminución inicial significativa, al aumentar el número de hilos de 1 a 3, el tiempo de ejecución se reduce drásticamente. Esto indica un aprovechamiento inicial eficiente del paralelismo. 

A partir de 3 hilos, la mejora en el tiempo de ejecución se desacelera y muestra una pendiente menos pronunciada.

Ya entre 6 y 8 hilos, la mejora adicional es menor, presentando  una especie de saturación. Sugiriendo que el sistema está siendo dominado por la parte serial, restándole así importancia a la cantidad de unidades de procesamiento. 

## Parte II

In [ ]:
#include <iostream>
#include <vector>
#include <mpi.h>

Se incluyen las bibliotecas estándar `iostream` y `vector` para la gestión de entrada/salida y vectores dinámicos, respectivamente. Así como `mpi.h` para habilitar las funciones de la interfaz de paso de mensajes MPI, que se usa para la paralelización.

In [ ]:
int main(int argc, char *argv[]) {
    if (argc != 3) {
        std::cerr << "Uso: " << argv[0] << " --n [vector_size]" << std::endl;
        return 1;
    }
    int n = atoi(argv[2]);
    if (n <= 0) {
        std::cerr << "Error: el tamaño del vector debe ser un número positivo." << std::endl;
        return 1;
    }

En el `main`, el programa recibe el tamaño del vector `n` como argumento de la línea de comandos, y valida que `n` sea un número positivo (de lo contrario muestra un mensaje de error y termina).

In [ ]:
    MPI_Init(&argc, &argv);
    int size, rank;
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

Se inicializa el entorno MPI con `MPI_Init`. Se obtienen los identificadores y el número de procesos.

In [ ]:
    double time_start = MPI_Wtime(); // Tiempo inicial

    int nlocal = n / size; // Tamaño base para cada proceso
    int rest = n % size;   // Elementos sobrantes a distribuir
    if (rest && (rank < rest))  
        nlocal++;

Se comienza a medir el tiempo para calcular el rendimieno de la aplicación.

Se divide el trabajo (tamaño del vector) equitativamente entre los procesos (`nlocal`).
Si hay elementos sobrantes (`rest`), se asignan a los procesos con identificadores más bajos (`rank < rest`).

In [ ]:
    int start = rank * nlocal; // Índice inicial del segmento
    if (rest && (rank >= rest)) 
        start += rest;
    int end = start + nlocal; // Índice final (no inclusivo) del segmento

Para  distribuir los datos de manera eficiente entre los procesos, y garantizar que el trabajo esté balanceado; se calcula el rango de índices (`start` a `end`) que cada proceso manejará en el vector global.

In [ ]:
    std::vector<double> x(nlocal, 1.0);
    std::vector<double> y(nlocal, -1.0);
    double alpha = 2.5;

Se crean vectores locales `x` y `y` para cada proceso, llenándolos con valores iniciales (`1.0` para `x` y `-1.0` para `y`).

Se define el escalar $\alpha$ usado en la operación AXPY.

In [ ]:
    for (int i = 0; i < nlocal; ++i) {
        y[i] += alpha * x[i];
    }

Cada proceso realiza la operación AXPY ($y = y+ \alpha \cdot x$) en su porción del vector.

In [ ]:
    //std::cout << "Proceso " << rank << ". Vector y local: y = [";
    //for (int i = 0; i < nlocal; ++i) {
        //std::cout << y[i];
        //if (i < nlocal - 1) std::cout << ", "; // Evitar la coma final
    //}
    //std::cout << "]" << std::endl;

    double time_end = MPI_Wtime(); // Tiempo final

Se imprimen los resultados locales desde cada proceso. Y, se establece el tiempo final.

In [ ]:
    if (rank == 0) {
        std::cout << "Tiempo total (s): " << time_end - time_start << std::endl;
    }

El proceso maestro (`rank == 0`) imprime el tiempo total de ejecución.

In [ ]:
    MPI_Finalize();
    return 0;
}

Finalmente, se libera los recursos asignados al entorno MPI con `MPI_Finalize`.

### Cálculo de Memoria Requerida por Proceso

La cantidad de elementos por proceso corresponde a 
\begin{equation}
n_{\text{local}} = \frac{n}{p}\approx \frac{5 \times 10^8}{p}
\end{equation}

Si hay elementos sobrantes, algunos procesos tendrán $ n_{\text{local}} + 1 $, pero se ignorará este caso ya que no modifica gravemente el resultado.


Luego, cada proceso tiene dos vectores locales, por lo tanto:
\begin{equation}
\text{Memoria total por proceso} = 2 \times n_{\text{local}} \times \text{tamaño de un elemento}
\end{equation}

\begin{equation}
\Rightarrow \text{Memoria total por proceso} = 2 \times \frac{5 \times 10^8}{p} \times 8 \, \text{bytes}
\end{equation}

Para ejemplificar, si solo se tiene un proceso, la memoria requerida por proceso es de 8GB, y si se tiene ocho procesos, es de 2GB.

Por último, se presenta el gráfico de escalabilidad del código utilizando hasta 8 procesos:

<div style="text-align: center;">
  <img src="axpy.png" style="width: 80%; margin: auto;">
</div>